In [3]:
import pandas as pd
stocks = pd.read_csv('/scratch/arz8448/deeprl-hw/Project/data/all_stocks_5yr.csv')
fixed_names = {k:v for v, k in zip(stocks.columns[1:], stocks.columns[:-1])}
fixed_names['index']='date'
fixed_names['volume'] = 'adj_close'

stocks = stocks.reset_index().rename(columns=fixed_names).sort_values(['Name','date'])
print(f'There are {stocks.Name.nunique()} stocks')
print(stocks.date.max())
stocks.to_csv('/scratch/arz8448/deeprl-hw/Project/data/all_stocks_5yr.csv', index=False)
stocks = pd.read_csv('/scratch/arz8448/deeprl-hw/Project/data/all_stocks_5yr.csv')
stocks.head()

There are 466 stocks
2021-10-18


,date,open,high,low,close,volume,adj_close,Name
0,2016-10-18,45.830002,45.270000,45.700001,45.439999,1949100.0,43.582001,A
1,2016-10-19,46.150002,45.340000,45.410000,46.020000,1769000.0,44.138287,A
2,2016-10-20,46.220001,45.650002,46.000000,46.049999,1493600.0,44.167057,A
3,2016-10-21,45.669998,45.290001,45.669998,45.570000,1011300.0,43.706684,A
4,2016-10-24,45.950001,45.639999,45.880001,45.849998,1666400.0,43.975243,A


In [4]:
tickers = ['AAPL', 'AMZN', 'BA', 'GOOGL', 'JPM', 'KO', 'MSFT', 'SPG', 'STT', 'TGT',
           'UNH', 'VMC', 'VZ', 'WMT', 'XEL', 'XOM']
all([t in stocks.Name.unique() for t in tickers])

True

In [5]:
import numpy as np
open_vals = []
high_vals = []
low_vals = []
close_vals = []

for ticker in tickers:
    print(ticker)
    df = stocks[stocks.Name == ticker]
    stock_opens = df['open']
    # Open(t) / Open(t-1)
    open_vals.append(stock_opens.iloc[1:].reset_index(drop = True) / 
                     stock_opens.iloc[:-1].reset_index(drop = True))
    # High(t-1) / Open(t-1) and so fort
    high_vals.append(df['high'].iloc[:-1] / stock_opens.iloc[:-1])
    low_vals.append(df['low'].iloc[:-1] / stock_opens.iloc[:-1])
    close_vals.append(df['close'].iloc[:-1] / stock_opens.iloc[:-1])
    
data = np.array([close_vals, high_vals, low_vals, open_vals])

AAPL
AMZN
BA
GOOGL
JPM
KO
MSFT
SPG
STT
TGT
UNH
VMC
VZ
WMT
XEL
XOM


In [6]:
data.shape

(4, 16, 1258)

In [7]:
with open('./data.npy','wb') as f:
    np.save(f, data)

4 = Number of features
16 = Number of stocks being studied
1258 = Number of timesteps

In [10]:
with open('./data.npy','rb') as f:
    data = np.load(f)
data

array([[[0.99373997, 0.99456522, 0.99727381, ..., 0.99653472,
         0.9991659 , 0.99958594],
        [0.99358748, 0.99225953, 0.9966857 , ..., 0.99478743,
         0.99646414, 0.99238942],
        [0.97680936, 0.98501472, 0.99314696, ..., 0.98909257,
         0.99886674, 0.99259463],
        ...,
        [0.99328196, 0.99506222, 0.99132539, ..., 0.99883289,
         0.99865352, 0.98977465],
        [0.99289736, 0.994328  , 0.99360388, ..., 0.9984494 ,
         0.99407476, 0.9989394 ],
        [0.9937403 , 0.99859113, 0.99055938, ..., 0.99701946,
         0.99592572, 0.9898348 ]],

       [[0.99357075, 0.96637229, 0.99105473, ..., 0.98444132,
         0.98352787, 0.99040718],
        [0.98912167, 0.98895844, 0.98971427, ..., 0.98077862,
         0.98312435, 0.99001108],
        [0.97619678, 0.97395417, 0.97718667, ..., 0.97969366,
         0.97178153, 0.98922862],
        ...,
        [0.98893497, 0.99446963, 0.9864838 , ..., 0.98930171,
         0.98403536, 0.9850407 ],
        [0.9

In [17]:
stocks.groupby('Name').volume.mean().sort_values(ascending=False).iloc[:15]

Name
AMZN     1975.764407
GOOGL    1356.926354
GOOG     1355.439049
AZO       968.762383
CMG       783.023304
MTD       777.720873
EQIX      524.079348
BLK       503.959990
CHTR      441.602613
REGN      431.489134
TDG       395.766216
ORLY      365.179699
NFLX      342.518697
ROP       321.464514
LMT       315.719416
Name: volume, dtype: float64